In [1]:
#Names: Kyle, Bekhruz, R
#DO NOT EDIT THIS PART#
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torchvision import datasets, transforms
from PIL import Image
import matplotlib.pyplot as plt

# GPUs are 3x faster than CPU. Better to use if it is available 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define Loss Function
loss_function = nn.CrossEntropyLoss()

# This function returns the number of parameters in the model
def num_params(model):
  return sum([p.numel() for p in model.parameters()])

# Define a Training Function. This function will: compute the forward pass, backpropagate,
# update the weights, and repeat the steps for a given number of epochs. At each epoch, 
# it will output the training loss and test loss at every step
def train(epochs, model, trainloader, testloader, optimizer, loss_function):
  for epoch in range(epochs):
    loss_epoch = np.array([])
    train_correct, train_total = 0, 0
    test_correct, test_total = 0, 0

    for data, labels in trainloader:
      # convert into GPU objects if needed
      input_data = data.to(device)
      labels = labels.to(device)

      # forward pass
      predict = model(input_data)
      
      # backward pass
      loss = loss_function(predict, labels)
      optimizer.zero_grad()
      loss.backward()

      # update parameters (weights and biases)
      optimizer.step()

      # store progress
      loss_epoch = np.append(loss_epoch, loss.item())

    # evaluate test accuracy
    for data, labels in testloader:
      input_data = data.to(device)
      labels = labels.to(device)
      predict = model(input_data)
      for i, out in enumerate(predict):
        pred = torch.argmax(out)
        if pred == labels[i]:
          test_correct+=1
        test_total+=1

    test_accuracy = test_correct/test_total    
  
    print('epoch [{}/{}], training loss:{:.4f}, test accuracy:{:.4f}'.format(epoch+1, epochs, np.mean(loss_epoch), test_accuracy))
################################################################################################################################


In [2]:
# download and load data
batch_size = 256

# download and transform train dataset
train_loader = torch.utils.data.DataLoader(datasets.MNIST('./mnist_data', download=True, train=True, transform=transforms.Compose([
                                                transforms.ToTensor(), # first, convert image to PyTorch tensor
                                                transforms.Normalize((0.1307,), (0.3081,)) # normalize inputs
                                                ])), batch_size=batch_size, shuffle=True)

# download and transform test dataset
test_loader = torch.utils.data.DataLoader(datasets.MNIST('./mnist_data', download=True, train=False, transform=transforms.Compose([
                                                              transforms.ToTensor(), # first, convert image to PyTorch tensor
                                                              transforms.Normalize((0.1307,), (0.3081,)) # normalize inputs
                                                          ])), batch_size=batch_size, shuffle=True)

In [3]:
# it is a good idea to take a look at the data. Here we see it is a 28x28 grayscale image
for data, labels in train_loader:
  print(data[0].size())
  break

torch.Size([1, 28, 28])


In [4]:
####                #### 
#### EDIT THIS CELL ####
####                ####
########################


learning_rate = 0.001
n_epochs = 10

# neural network
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    ### Define Layers
    self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
    self.conv2 = nn.Conv2d(32, 64, 3, padding = 'valid')
    self.conv3 = nn.Conv2d(64, 64, 3, padding = 'valid')
    self.mp1 = nn.MaxPool2d(2)
    self.fc1 = nn.Linear(1024, 64)
    self.fc2 = nn.Linear(64, 10)
    #self.mp1 = nn.MaxPool2d(2)

  def forward(self, x):
    x = self.conv1(x)
    x = F.relu(x)
    x = self.mp1(x)

    x = self.conv2(x)
    x = F.relu(x)
    x = self.mp1(x)

    x = self.conv3(x)
    x = F.relu(x)

    x = torch.flatten(x, 1)

    x = self.fc1(x)
    x = F.relu(x)

    x = self.fc2(x)
    x = F.softmax(x)
    return x

  def predict(self, test_images):

        count = test_images.shape[0]
        result_np = []

        for idx in range(0, count):

            img = test_images[idx, :, :, :]
            img = np.expand_dims(img, axis=0)
            img = torch.Tensor(img).to(device)

            pred = self(img)
            pred_np = pred.cpu().detach().numpy()
            for elem in pred_np:
                result_np.append(elem)

        return result_np

  
# Every time you edit the neural network, you'll have to update this cell
# Create model object
model = NeuralNetwork().to(device)

# Loads Adam optimizer, which implements a version of gradient descent
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [5]:
# check the structure of your network
print(model)

# apply your model to a single input. This helps check that 
# the dimensions are correct
model(torch.rand(1,1,28,28, device=device))

NeuralNetwork(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=valid)
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=valid)
  (mp1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1024, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=10, bias=True)
)


C:\Users\Ravin\AppData\Local\Temp\ipykernel_3472\2870345730.py:41: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.softmax(x)


tensor([[0.0937, 0.1100, 0.0999, 0.0993, 0.1085, 0.0958, 0.1084, 0.0997, 0.0839,
         0.1009]], grad_fn=<SoftmaxBackward0>)

In [6]:
train(10, model, train_loader, test_loader, optimizer, loss_function)

C:\Users\Ravin\AppData\Local\Temp\ipykernel_3472\2870345730.py:41: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.softmax(x)


epoch [1/10], training loss:1.6579, test accuracy:0.8792
epoch [2/10], training loss:1.4961, test accuracy:0.9814
epoch [3/10], training loss:1.4802, test accuracy:0.9864
epoch [4/10], training loss:1.4770, test accuracy:0.9856
epoch [5/10], training loss:1.4741, test accuracy:0.9871
epoch [6/10], training loss:1.4729, test accuracy:0.9864
epoch [7/10], training loss:1.4728, test accuracy:0.9885
epoch [8/10], training loss:1.4708, test accuracy:0.9874
epoch [9/10], training loss:1.4700, test accuracy:0.9886
epoch [10/10], training loss:1.4694, test accuracy:0.9907


--------------
                                                            Saving the model
------------------


In [7]:
PATH = r'C:\Users\Ravin\Desktop\cs\python\courses\maths_in_ai\course_materials\final_projects\models\model1.pth'
torch.save(model.state_dict(), PATH)

----------------------------------------
                                                            Loading a model
---------------------------------------

In [8]:
model = NeuralNetwork()
model.load_state_dict(torch.load(PATH))
model.eval()

NeuralNetwork(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=valid)
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=valid)
  (mp1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1024, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=10, bias=True)
)

In [9]:
x = datasets.MNIST('./mnist_data', download=False, train=False, transform=transforms.Compose([
                                                              transforms.ToTensor(), # first, convert image to PyTorch tensor
                                                              transforms.Normalize((0.1307,), (0.3081,))]))

In [10]:
num_samples = 20

for data, label in x:
    print(f'{torch.argmax(torch.Tensor(model.predict(data)))} : {label}')

IndexError: too many indices for tensor of dimension 3